## Example of how to run detections using the original EQTransformer trained model
Resource used: https://eqtransformer.readthedocs.io/en/latest/tutorial.html

# Environment/repository set-up

In [ ]:
import numpy as np
import pandas as pd
import shutil
import os
from zipfile import ZipFile
import glob
import glob2 as glob
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import obspy

In [ ]:
# Install and set-up EQTransformer following installation via git

# NOTE: if using Google CoLab, make sure the runtime includes GPU before starting
!pip install git+https://github.com/smousavi05/EQTransformer

In [ ]:
# Clone in the repository setup- this is what contains the pre-trained model, so this is necessary

!git clone https://github.com/UW-ESS-DS/krauss-repo

# Download Data

In [ ]:
# Station list needs to be in json format- here's an example of how one is made

# Make list of stations to query

from EQTransformer.utils.downloader import makeStationList

# For all stations:
makeStationList('stationlist.json',client_list=["IRIS"],min_lat =47.5 , max_lat = 48.5,min_lon=-129.4,max_lon=-128.8,start_time='2018-08-03T00:00:00',end_time='2018-08-05T00:00:00',channel_list=['HH[ZNE]','EH[ZNE]', 'HH[Z21]','EH[Z21]', 'CH[ZNE]'],filter_network=['SY'])

In [ ]:
# Download mseeds for desired time period - this function stores them in a specific folder/file tree format that is needed for the other EQT commands
from EQTransformer.utils.downloader import downloadMseeds

stime = "2018-08-01 00:00:00.00"
ftime = "2018-08-05 00:00:00.00"

downloadMseeds(client_list=["IRIS"], stations_json='stationlist.json', output_dir="downloads_mseeds_august3_2018",min_lat =47.5 , max_lat = 48.5,min_lon=-129.4,max_lon=-128.8,start_time=stime,end_time=ftime, chunk_size=1, channel_list=["HH[ZNE]","EH[ZNE]", "HH[Z21]","EH[Z21]", "CH[ZNE]"], n_processor=1)

# Perform detections

In [ ]:
from EQTransformer.core.mseed_predictor import mseed_predictor
    
mseed_predictor(input_dir='/home/zkrauss/scratch/krauss-repo/downloads_mseeds',   
                    input_model='/home/zkrauss/scratch/krauss-repo/eq_project/eqtransformer_local/ModelsAndSampleData/EqT_model.h5',
                    stations_json='end_stationlist.json',
                    output_dir='/home/zkrauss/scratch/krauss-repo/end_test1',
                    detection_threshold=0.2,                
                    P_threshold=0.1,
                    S_threshold=0.1,  
                    number_of_plots=0,
                    batch_size=20,
                    overlap=0.3,
                    output_probabilities=True)

# Look at results- both picks and probability streams

## Detections and picks are stored in csv format

In [ ]:
# You can gather them into a pandas dataframe as follows, just change your base directory
base_dir = '/home/zkrauss/alaska-continuous/krauss-repo/aacse_detection_all/'
res_files = glob.glob(base_dir + '**/X*results.csv')
eqt_picks = pd.concat([pd.read_csv(f) for f in res_files])

## Probability streams are stored in h5

In [ ]:
import h5py

# To get a list of earthquakes found for a specific station, you can do as follows for a given station file
filename='/home/zkrauss/alaska-continuous/krauss-repo/aacse_detection_forfigs/EP22_outputs/prediction_probabilities.hdf5'
f = h5py.File(filename, 'r')
list(f.keys())


In [ ]:
# Then pick out an earthquake from the list and you can get the detection, P, and S probability vectors as follows

dset = f['2019-05-27 03:15:18.009998']
d_prob = dset['Earthquake'][:]
p_prob = dset['P_arrival'][:]
s_prob = dset['S_arrival'][:]